In [2]:
import numpy as np
import pandas

In [3]:
### load file as np array
def load(file_name):
    genotype = []
    data = pandas.read_csv(file_name, dtype = object, header = None)[0]
#    f = open(file_name, 'r+b')
#     for line in f:
#         print(line)
#         n = list(line)
#         print(n)
#         genotype.append(n)
#    f.close()
#    genotype = np.array(, dtype=np.int8)
    for line in data:
        genotype.append(list(line))
    genotype = np.array(genotype, dtype=np.int8)
    return genotype

veasy = load("data/very_easy_training_genotypes.txt")
easy = load("data/easy_training_genotypes.txt")
med = load("data/medium_training_genotypes.txt")
hard = load("data/hard_training_genotypes.txt")
veasy_theirs = load("data/very_easy_training_haplotypes.txt")
easy_theirs = load("data/easy_training_haplotypes.txt")
medium_theirs = load("data/medium_training_haplotypes.txt")
hard_theirs = load("data/hard_training_haplotypes.txt")

In [4]:
### sorts rows of a and removes duplicate rows
def unique_rows(a):
    a = np.ascontiguousarray(a)
    unique_a = np.unique(a.view([('', a.dtype)]*a.shape[1]))
    return unique_a.view(a.dtype).reshape((unique_a.shape[0], a.shape[1]))

### sort their haplotypes to compare
def check(theirs, str):
    theirs = unique_rows(theirs)
    np.savetxt(str,theirs, fmt = '%s', delimiter="")

In [6]:
def cartesian(arrays, out=None):
    arrays = [np.asarray(x) for x in arrays]
    dtype = arrays[0].dtype

    n = np.prod([x.size for x in arrays])
    if out is None:
        out = np.zeros([n, len(arrays)], dtype=dtype)

    m = n / arrays[0].size
    out[:,0] = np.repeat(arrays[0], m)
    if arrays[1:]:
        cartesian(arrays[1:], out=out[0:m,1:])
        for j in range(1, arrays[0].size):
            out[j*m:(j+1)*m,1:] = out[0:m,1:]
    return out

### Takes a matrix, 'a', of genotype reads
### and saves the resulting haplotype
### phases to the file 'str'
### provides all possible haplotypes
def trivial_method(a, str):
    haplotypes = []
    haplotype = []
    delete = []
    out = np.empty([1, a.shape[1]])
    
    for x in range(len(a)):
        line = a[x]
        for i in range(len(line)):
            if line[i] == 1:
                break
            elif line[i] == 0:
                haplotype.append(0)
            else:
                haplotype.append(1)
        if len(haplotype) == len(line):
            haplotypes.append(haplotype)
            delete.append(x)
        haplotype = []
    a = np.delete(a, delete, 0)
    for x in range(len(a)):
        line = a[x]
        for i in range(len(line)):
            if line[i] == 1:
                haplotype.append([0,1])
            elif line[i] == 0:
                haplotype.append([0])
            else:
                haplotype.append([1])
        haplotypes.append(haplotype)
        haplotype = []
    haplotypes = np.array(haplotypes)
    for haplotype in haplotypes:
        line = cartesian(haplotype)
        out = np.append(out, line, axis=0)
    out = np.delete(out, 0, 0)
    out = np.array(out, dtype=np.int8)
    out = unique_rows(out)
    np.savetxt(str, out, fmt = '%s', delimiter="")

trivial_method(veasy, "trivial_veasy.txt")
trivial_method(easy, "trivial_easy.txt")
# trivial_method(med, "trivial_medium.txt")
# trivial_method(hard, "trivial_hard.txt")

    

ValueError: setting an array element with a sequence.

In [6]:
### Takes a matrix, 'a', of genotype reads
### and saves the resulting haplotype
### phasing to the file 'str'
### Provides optimum minimum parsimony
def imp_method(a, str):
    haplotypes = []
    haplotype = []
    delete = []
    # first get known haplotypes
    for x in range(len(a)):
        line = a[x]
        for i in range(len(line)):
            if line[i] == 1:
                break
            elif line[i] == 0:
                haplotype.append(0)
            else:
                haplotype.append(1)
        if len(haplotype) == len(line):
            haplotypes.append(haplotype)
            delete.append(x)
        haplotype = []    
    haplotypes = np.array(haplotypes)
    haplotypes = unique_rows(haplotypes)
    a = np.delete(a, delete, 0)

    # then go through improved clark's method's iterations
    while len(a) > 0:
        for haplotype in haplotypes:
            diff = a - haplotype
            zero_rows = np.where(np.all(diff >= 0,axis=1))[0]
            two_rows = np.where(np.all(diff < 2,axis=1))[0]
            rows = np.intersect1d(zero_rows, two_rows)
            haplotypes = np.append(haplotypes, diff[rows], axis=0)
            haplotypes = unique_rows(haplotypes)
            a = np.delete(a, rows, 0)
    np.savetxt(str,haplotypes, fmt = '%s', delimiter="")

In [7]:
%time imp_method(veasy, "mine_veasy.txt")
check(veasy_theirs, "theirs_veasy.txt")
%time imp_method(easy, "mine_easy.txt")
check(easy_theirs, "theirs_easy.txt")
%time imp_method(med, "mine_medium.txt")
check(medium_theirs, "theirs_medium.txt")
%time imp_method(hard, "mine_hard.txt")
check(hard_theirs, "theirs_hard.txt")

CPU times: user 4.77 ms, sys: 555 µs, total: 5.33 ms
Wall time: 4.92 ms
CPU times: user 107 ms, sys: 1.4 ms, total: 108 ms
Wall time: 108 ms
CPU times: user 346 ms, sys: 557 µs, total: 346 ms
Wall time: 346 ms
CPU times: user 12.1 s, sys: 53.1 ms, total: 12.1 s
Wall time: 12.1 s
